## Preprocessing Practice with Zomato Dataset
<br>

In [ ]:
# Load the numpy and pandas libraries with alias 'np' and 'pd' into your namespace 
import numpy as np
import pandas as pd
# and import only pyplot from matplotlib library as 'plt' into your namespace
from matplotlib import pyplot as plt

# Import the content of a .csv file into a dataFrame
df = pd.read_csv('zomato_kisa.csv', index_col = None, encoding = 'utf-8')

In [ ]:
# Display the first 5 entries
df.head() 

In [ ]:
# Display the number of rows and columns
df.shape

In [ ]:
# Just some modifications to ease our acces to dataframe columns
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

df.head()

In [ ]:
# Display a quick statistic summary of your data (based on quantitative features)
df.describe()

In [ ]:
# Display a quick statistic summary of your data (based on qualitative features)
df.describe(include = "O")

In [ ]:
# Restrict the number of rows to display
pd.set_option('max_rows', 10)
#pd.reset_option('max_rows')   # you can always reset it

In [ ]:
# Count of "cuisines" is 95, let's check the feature
df.cuisines

# or 
#df.cuisines.tail(20)   # (just because I know the data)

In [ ]:
# The proper way to check it: If there is NaN, it returns True

df.cuisines.isna()

In [ ]:
# The proper way to check it: If there is NaN, it returns True
# When you add any(), if there is at least one True, then it returns True
df.cuisines.isna().any()

In [ ]:
# One solution is to drop the column with the NaN entries 
df_dropcol = df.dropna(axis='columns')
df_dropcol.shape

In [ ]:
# Another solution is to drop the rows with the NaN entries
df_droprow = df.dropna(axis='rows')
df_droprow.shape

In [ ]:
# Or 
# First, let's check the rows with NaN entries
df[df.cuisines.isna()]

In [ ]:
# We can always fill them manually using their index
df.iloc[84,9] = "American"

In [ ]:
# Check visually if the index = 84 is replaced
pd.reset_option('max_rows')
df.cuisines.tail(20)

In [ ]:
# Or we can use fillna() method
df.cuisines.fillna("American", inplace=True)

In [ ]:
# Check the new values of NaN 
df.cuisines.tail(20)

<br>
<b>Redundancy check
<br>

In [ ]:
# check if there is any redundancy (if there is any duplicated entry it returns True)
df.duplicated().any()

<br>
<b> Dataset Mess-up
<br>

In [ ]:
# First we copy our dataset
df_copy = df

Let's add a new quantitative feature: "revenue" to our dataset

In [ ]:
# First generate a numpy array and fill it randomly
rev = np.random.randint(10,1000, size = len(df_copy))*100

In [ ]:
# Create a new column for "revenue" in df_copy
df_copy['revenue'] = rev
df_copy.head()

<br>
Let's add some NaN values in the following columns: 
<ul>
<li>"revenue"</li>
<li>"rating_color"</li>
<li>"rating_text"</li>
</ul>

In [ ]:
# Generate some random indexes for "revenue" feature
ind_revenue = np.random.randint(len(df_copy), size = 5)
print(type(ind_revenue), ind_revenue)

In [ ]:
# Generate some random indexes for "rating_color" feature
ind_color = np.random.randint(len(df_copy), size = 5)
print(ind_color)

In [ ]:
# Generate some random indexes for "rating_text" feature
ind_text = np.random.randint(len(df_copy), size = 5)
print(ind_text)

In [ ]:
# Fill them with NaN values in df_copy
for i in range(len(ind_revenue)):
    df_copy.loc[ind_revenue[i], 'revenue'] = np.nan
    df_copy.loc[ind_color[i], 'rating_color'] = np.nan
    df_copy.loc[ind_text[i], 'rating_text'] = np.nan

In [ ]:
# Check for the modifications
df_copy.isna().any()

<br>
<b>Decision to make: Drop them or replace/fill in the values?

<br>

In [ ]:
# List the rows with NaN values for "revenue"
df_copy[df_copy.revenue.isna()]

In [ ]:
# Check for the descriptive stats
df_copy.revenue.describe()

In [ ]:
# and its mode
df_copy.revenue.mode()

In [ ]:
# Anatomy of type conversion :)
r_mean = df_copy.revenue.mean()
r_mean_rounded_up = np.around(df_copy.revenue.mean())
r_int = int(r_mean_rounded_up)  # ----> Type conversion
print(r_int)

# Or in a single line
#print(int(np.around(df_copy.revenue.mean())))

In [ ]:
# Fill in the NaN values with mean, median, mode, etc
df_copy.revenue.fillna(int(np.around(df_copy.revenue.mean())), inplace = True)

In [ ]:
# Let's check if everything is OK
df_copy.loc[ind_revenue, ['restaurant_id','restaurant_name','revenue']]

<br>
Let's look at the qualitative features "rating_color" and "rating_text"

In [ ]:
# List the rows with NaN values for "rating_color"
df_copy[df_copy.rating_color.isna()]

In [ ]:
# List the rows with NaN values for "rating_text"
df_copy[df_copy.rating_text.isna()]

<br>
If we look closely, we can that there is a rule-based relationship between the following columns:
<ul>
<li>"aggregate_rating"</li>
<li>"rating_color"</li>
<li>"rating_text"</li>
</ul>

Such as: 

> 0.0 ---> White ---> Not Rated

> 1.5 ---> Red ---> Poor

> 2.5 ---> Orange ---> Average

> 3.5 ---> Yellow ---> Good

> 4.0 ---> Green ----> Very Good

> 4.5 ---> Dark Green ---> Excellent

Then let's fill in the NaN values using a for loop

In [ ]:
# 0.0 -----> White ------> Not Rated
# 1.5 -----> Red --------> Poor
# 2.5 -----> Orange -----> Average
# 3.5 -----> Yellow -----> Good
# 4.0 -----> Green ------> Very Good
# 4.5 ---> Dark Green ---> Excellent

for i in range(len(df_copy.aggregate_rating)):
    if (pd.isna(df_copy.loc[i,'rating_color']) or pd.isna(df_copy.loc[i,'rating_text'])):
        if (df_copy.loc[i,'aggregate_rating'] == 0):
            df_copy.loc[i,'rating_color'] = "White"
            df_copy.loc[i,'rating_text'] = "Not Rated"
        elif (df_copy.loc[i,'aggregate_rating'] >= 1.5 and df_copy.loc[i,'aggregate_rating'] < 2.5):
            df_copy.loc[i,'rating_color'] = "Red"
            df_copy.loc[i,'rating_text'] = "Poor"
        elif (df_copy.loc[i,'aggregate_rating'] >= 2.5 and df_copy.loc[i,'aggregate_rating'] < 3.5):
            df_copy.loc[i,'rating_color'] = "Orange"
            df_copy.loc[i,'rating_text'] = "Average"
        elif (df_copy.loc[i,'aggregate_rating'] >= 3.5 and df_copy.loc[i,'aggregate_rating'] < 4):
            df_copy.loc[i,'rating_color'] = "Yellow"
            df_copy.loc[i,'rating_text'] = "Good"
        elif (df_copy.loc[i,'aggregate_rating'] >= 4 and df_copy.loc[i,'aggregate_rating'] < 4.5):
            df_copy.loc[i,'rating_color'] = "Green"
            df_copy.loc[i,'rating_text'] = "Very Good"
        elif (df_copy.loc[i,'aggregate_rating'] >= 4.5):
            df_copy.loc[i,'rating_color'] = "Dark Green"
            df_copy.loc[i,'rating_text'] = "Excellent"
        else:
            df_copy.loc[i,'rating_color'] = "None"
            df_copy.loc[i,'rating_text'] = "None"

In [ ]:
# If you want to check the values, you can display them
df_copy.loc[ind_color, ['aggregate_rating','rating_color','rating_text']]

In [ ]:
# Let's make a final check
df_copy.isna().any()

In [ ]:
# Now, it's your turn, you can re-do all the things above with the full zomato dataset

In [ ]:
# ...